In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession with Hive support
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
# Execute a query on a Hive table
result_df = spark.sql("SELECT count(*) FROM twitter_data.tweets ")

# Show the results
result_df.show()

+--------+
|count(1)|
+--------+
|   14300|
+--------+



In [4]:
firstRows_DF=spark.sql("select * from twitter_data.tweets limit 20 ")
firstRows_DF.show()

+-------------------+--------------------+-------------------+-------------+-----------+----------+-----------+-------------------+--------------------+---------------+---------------+---------------+-----------+------------+--------+----+-----+---+----+
|                 id|                text|          author_id|retweet_count|reply_count|like_count|quote_count|         created_at|                name|       username|followers_count|following_count|tweet_count|listed_count|verified|year|month|day|hour|
+-------------------+--------------------+-------------------+-------------+-----------+----------+-----------+-------------------+--------------------+---------------+---------------+---------------+-----------+------------+--------+----+-----+---+----+
|1654239177595146240|RT : Omdurman Mid...| 711486983655301121|           26|          0|         0|          0|2023-05-04 21:38:48|              Hisham|    hishamjr145|            335|            376|      10988|           2|   false|2

In [5]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS twitter_data.users_dim (
        author_id STRING,
        id STRING,
        name STRING,
        username STRING,
        followers_count INT,
        following_count INT,
        tweet_count INT,
        listed_count INT,
        verified STRING
    )
    PARTITIONED BY (year INT, month INT, day INT)
    STORED AS PARQUET
    LOCATION '/FileStore/twitter_data/users_dim'
""")


""


In [6]:

spark.sql("""
    INSERT OVERWRITE TABLE twitter_data.users_dim
    PARTITION (year, month, day)
    SELECT author_id, id, name, username, followers_count, following_count, tweet_count, listed_count, verified, year(created_at) AS year, month(created_at) AS month, day(created_at) AS day
    FROM twitter_data.tweets
""")

""


In [7]:
spark.sql("""
select * from twitter_data.users_dim
limit 3
""")

author_id,id,name,username,followers_count,following_count,tweet_count,listed_count,verified,year,month,day
1496586301486809094,1654239162386591745,🔜Samri,Samrawitkalayu7,3968,1492,646616,2,false,2023,5,4
1573068165524996096,1654239161543454726,Tsige Geberhiwot,TsigeG21,1055,1130,69471,0,false,2023,5,4
1492197849760677897,1654239159106584578,Aluma🦋,AlumaAlumaa,5318,1229,382434,0,false,2023,5,4


In [8]:
spark.sql("""CREATE EXTERNAL TABLE IF NOT EXISTS twitter_data.tweets_dim (
    id STRING ,
    text STRING,
    author_id STRING,
    retweet_count INT,
    reply_count INT,
    like_count INT,
    quote_count INT,
    created_at TIMESTAMP
   
) PARTITIONED BY (year INT, month INT, day INT, hour INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweets_dim';
""")


""


In [9]:
spark.sql("""
INSERT OVERWRITE TABLE twitter_data.tweets_dim
PARTITION (year, month, day, hour)
SELECT id, text, author_id, retweet_count, reply_count, like_count, quote_count, created_at, year(created_at) as year, month(created_at) as month, day(created_at) as day, hour(created_at) as hour
FROM twitter_data.tweets
""")

""


In [10]:
spark.sql("""
select * from twitter_data.tweets_dim
limit 3
""")

id,text,author_id,retweet_count,reply_count,like_count,quote_count,created_at,year,month,day,hour
1654239162386591745,RT : Tigryan refu...,1496586301486809094,41,0,0,0,2023-05-04 21:38:45,2023,5,4,21
1654239161543454726,RT : After she fl...,1573068165524996096,24,0,0,0,2023-05-04 21:38:45,2023,5,4,21
1654239159106584578,RT : After she fl...,1492197849760677897,46,0,0,0,2023-05-04 21:38:44,2023,5,4,21


In [11]:
spark.sql("""CREATE EXTERNAL TABLE IF NOT EXISTS twitter_data.author_matrix (
    author_id STRING,
    tweet_id STRING
) PARTITIONED BY (year INT, month INT, day INT, hour INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/author_matrix';
""")

spark.sql("""INSERT OVERWRITE TABLE twitter_data.author_matrix
PARTITION (year, month, day, hour)
SELECT author_id, id as tweet_id, year(created_at) as year, month(created_at) as month, day(created_at) as day, hour(created_at) as hour
FROM twitter_data.tweets_dim;
""")


""


In [12]:
spark.sql("""
select * from twitter_data.author_matrix
limit 3
""")

author_id,tweet_id,year,month,day,hour
1496586301486809094,1654239162386591745,2023,5,4,21
1573068165524996096,1654239161543454726,2023,5,4,21
1492197849760677897,1654239159106584578,2023,5,4,21


In [13]:
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS twitter_data.time_dim (
    time_id INT,
    created_at TIMESTAMP
)
PARTITIONED BY (year INT, month INT, day INT, hour INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/time_dim';
""")

spark.sql("""
INSERT OVERWRITE TABLE twitter_data.time_dim
PARTITION (year, month, day, hour)
SELECT
ROW_NUMBER() OVER (ORDER BY created_at) AS time_id,
created_at, year, month,day,hour
FROM twitter_data.time_dim;
""")


""


In [14]:
spark.sql("""
select * from twitter_data.time_dim 
limit 3
""")

time_id,created_at,year,month,day,hour


In [15]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.tweets_more_than_50 (
    hour INT,
    tweets_count INT,
    earliest_created_at TIMESTAMP
) PARTITIONED BY (year , month , day )
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweets_more_than_50';
""")

spark.sql("""
INSERT OVERWRITE TABLE twitter_data.tweets_more_than_50
PARTITION (year, month, day)
SELECT
    hour,
    COUNT(*) AS tweets_count,
    year(created_at) AS year,
    month(created_at) AS month,
    day(created_at) AS day,
    MIN(created_at) AS earliest_created_at
FROM twitter_data.tweets_dim
GROUP BY hour, year, month, day
HAVING COUNT(*) > 50
""")


AnalysisException: partition column year is not defined in table twitter_data.tweets_more_than_50, defined table columns are: hour, tweets_count, earliest_created_at

In [16]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.tweet_matrix_raw (
    id String ,
    retweet_count INT,
    reply_count INT,
    like_count INT,
    quote_count INT
) PARTITIONED BY (year INT, month INT, day INT, hour INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweet_matrix_raw';
""")


""


In [17]:

spark.sql("""
INSERT OVERWRITE TABLE  twitter_data.tweet_matrix_raw
PARTITION (year, month, day, hour)
SELECT id, retweet_count, reply_count, like_count, quote_count, 
  year(created_at) as year, month(created_at) as month, day(created_at) as day, hour(created_at) as hour
FROM twitter_data.tweets;
""")

""


In [18]:
spark.sql("""
select * from twitter_data.tweet_matrix_raw
limit 3
""")

id,retweet_count,reply_count,like_count,quote_count,year,month,day,hour
1654239167377817603,65,0,0,0,2023,5,4,21
1654239164462669833,26,0,0,0,2023,5,4,21
1654239162386591745,41,0,0,0,2023,5,4,21


In [19]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.tweets_count_raw (
    hour INT,
    tweets_count INT
) PARTITIONED BY (year INT, month INT, day INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweets_count_raw';
""")


""


In [20]:
spark.sql("""
INSERT OVERWRITE TABLE twitter_data.tweets_count_raw
PARTITION (year, month, day)
SELECT 
  HOUR(CAST(t.created_at AS TIMESTAMP)) AS hour,
  COUNT(1) AS tweets_count,
  YEAR(CAST(t.created_at AS DATE)) AS year,
  MONTH(CAST(t.created_at AS DATE)) AS month,
  DAY(CAST(t.created_at AS DATE)) AS day
FROM twitter_data.tweets t
GROUP BY HOUR(CAST(t.created_at AS TIMESTAMP)), YEAR(CAST(t.created_at AS DATE)), MONTH(CAST(t.created_at AS DATE)), DAY(CAST(t.created_at AS DATE))
""")

""


In [21]:
spark.sql("""
select * from twitter_data.tweets_count_raw
limit 1
""")

hour,tweets_count,year,month,day
21,14300,2023,5,4


In [22]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.most_active_hour_raw (
    hour INT,
    tweets_count INT
) PARTITIONED BY (year INT, month INT, day INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/most_active_hour_raw';
""")





""


In [23]:
spark.sql("""
INSERT OVERWRITE TABLE twitter_data.most_active_hour_raw
PARTITION (year, month, day)
SELECT hour(created_at) AS hour,
COUNT(*) AS tweets_count,
YEAR(created_at) AS year,
MONTH(created_at) AS month,
DAY(created_at) AS day
FROM twitter_data.tweets
GROUP BY hour(created_at), YEAR(created_at), MONTH(created_at), DAY(created_at);
""")

""


In [24]:
spark.sql("""
select * from twitter_data.most_active_hour_raw
limit 3
""")

hour,tweets_count,year,month,day
21,14300,2023,5,4


In [25]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.tweets_more_than_50_raw (
    hour INT,
    tweets_count INT
) PARTITIONED BY (year INT, month INT, day INT)
STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweets_more_than_50_raw';
""")
spark.sql("""
INSERT OVERWRITE TABLE twitter_data.tweets_more_than_50_raw
PARTITION (year, month, day)
SELECT hour(created_at) as hour, COUNT(*) as tweets_count, 
  year(created_at) as year, month(created_at) as month, day(created_at) as day
FROM twitter_data.tweets
GROUP BY year(created_at), month(created_at), day(created_at), hour(created_at)
HAVING COUNT(*) > 50;
""")

""


In [26]:
spark.sql("""
select * from twitter_data.tweets_more_than_50_raw
WHERE tweets_count > 50
limit 3
""")


hour,tweets_count,year,month,day
21,14300,2023,5,4


In [27]:
spark.sql("""

CREATE TABLE IF NOT EXISTS twitter_data.tweet_count_hourly (
    year INT,
    month INT,
    day INT,
    hour INT,
    tweet_count INT
) STORED AS PARQUET
LOCATION '/FileStore/twitter_data/tweet_count_hourly';

""")



""


In [28]:
spark.sql("""

INSERT OVERWRITE TABLE twitter_data.tweet_count_hourly
SELECT
    year(created_at) AS year,
    month(created_at) AS month,
    day(created_at) AS day,
    hour(created_at) AS hour,
    COUNT(*) AS tweet_count
FROM twitter_data.tweets
GROUP BY year(created_at), month(created_at), day(created_at), hour(created_at);
""")


""


In [29]:
spark.sql("""
select * from twitter_data.tweet_count_hourly
limit 3
""")


year,month,day,hour,tweet_count
2023,5,4,21,14300


In [30]:
spark.sql("""
CREATE TABLE IF NOT EXISTS twitter_data.top_10_retweeted_tweets_fact (
    tweet_id string,
    text STRING,
    retweet_count INT
) STORED AS PARQUET
LOCATION '/FileStore/twitter_data/top_10_retweeted_tweets_fact';
""")


""


In [33]:
spark.sql("""
INSERT INTO twitter_data.top_10_retweeted_tweets_fact
SELECT CAST(id AS STRING) AS tweet_id, text, retweet_count
FROM twitter_data.tweets
ORDER BY retweet_count DESC
LIMIT 10;
""")
